# Reression Model : Advance House Price Prediction

In [1]:
## Import Libraiers
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error as MSE 

In [2]:
# import dataset train And Test
X = pd.read_csv('X_train')
y = pd.read_csv('y_train')

In [3]:
y

,Unnamed: 0,SalePrice
0,0,208500.0
1,1,181500.0
2,2,223500.0
3,3,140000.0
4,4,250000.0
...,...,...
2914,2914,163000.0
2915,2915,163000.0
2916,2916,163000.0
2917,2917,163000.0


In [4]:
y = y.drop(['Unnamed: 0'],axis=1)
X= X.drop(['Unnamed: 0'],axis=1)

In [5]:
print(X.shape)
print(y.shape)

(2919, 83)
(2919, 1)


In [6]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_nan,MasVnrArea_nan,BsmtFullBath_nan,GarageYrBlt_nan
0,0.235294,0.666667,0.418208,0.366344,1.0,1.0,0.000000,0.333333,1.0,0.00,...,1.0,0.0,0.090909,0.50,0.666667,0.75,0.0,0.0,0.0,0.0
1,0.000000,0.666667,0.495064,0.391317,1.0,1.0,0.000000,0.333333,1.0,0.25,...,1.0,0.0,0.363636,0.25,0.666667,0.75,0.0,0.0,0.0,0.0
2,0.235294,0.666667,0.434909,0.422359,1.0,1.0,0.333333,0.333333,1.0,0.00,...,1.0,0.0,0.727273,0.50,0.666667,0.75,0.0,0.0,0.0,0.0
3,0.294118,0.666667,0.388581,0.390295,1.0,1.0,0.333333,0.333333,1.0,0.50,...,1.0,0.0,0.090909,0.00,0.666667,0.00,0.0,0.0,0.0,0.0
4,0.235294,0.666667,0.513123,0.468761,1.0,1.0,0.333333,0.333333,1.0,0.25,...,1.0,0.0,1.000000,0.50,0.666667,0.75,0.0,0.0,0.0,0.0


## Train_Test_split Dataset

In [7]:
## Create Data Splitting in train and test data 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.6,random_state=0)

### Prediction And Selecting Alorithem

In [8]:
import xgboost
regressor=xgboost.XGBRegressor()

In [9]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]

In [10]:
## Hyper Parameter Optimization

n_estimators = [100,500,900,1100,1500]
max_depth = [2,3,5,10,15]
booster = ['gbtree','gblinear']
learning_rate = [0.05, 0.1, 0.15, 0.20]
min_child_weight=[1,2,3,4]

In [11]:
# Define Hyperparameter to Search
hyperparameter_grid ={
     'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
}

In [12]:
# Set up the random search with 4-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [13]:
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   14.8s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  2.1min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  4.3min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:  6.2min finished


RandomizedSearchCV(cv=5,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n...
                                          validate_parameters=None,
                                          verbosity=None),
                   n_iter=50, n_jobs=4,
                   para

In [14]:
random_cv.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=15,
             min_child_weight=4, missing=nan, monotone_constraints='()',
             n_estimators=900, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
regressor=xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=15, min_child_weight=4, missing=None, n_estimators=900,
       n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [16]:
regressor.fit(X_train,y_train)

[15:49:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[15:49:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=15,
             min_child_weight=4, missing=None, monotone_constraints='()',
             n_estimators=900, n_jobs=4, nthread=4, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, seed=0, silent=True, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
import pickle
filename = 'Ad_House_price_model.pkl'
pickle.dump(regressor, open(filename, 'wb'))

In [22]:
# Predict the model 
y_pred = regressor.predict(X_test) 

In [24]:
y_pred

array([149587.11, 131739.4 , 155533.72, ..., 171704.98, 193243.33,
       141898.25], dtype=float32)

In [32]:
##Create Sample Submission file and Submit using ANN
pred=pd.DataFrame(y_pred)
sub_df=pd.read_csv('sample_submission.csv')
datasets=pd.concat([sub_df['Id'],pred],axis=1)
datasets.columns=['Id','SalePrice']
datasets.to_csv('sample_submission.csv',index=False)